# "Bin" statistics & histogram

 Statistics "bin" are counted on the sample of a single continuous variable; firstly, it divides the 
 range of values into bins (sequential, non-overlapping sections), and then it counts the number of observations in each bin. 
 It's weighted; it means the weighted count for each bin is calculated (each element within a bin counted along with its weight).
 It's really important to carefully choose bin constructing method (for example, by exact number of bins or by their width). This decision has a big impact on how the data is shown and studied. It makes sure that the way the data is shown is easy to understand and gives a true picture of the information.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

Binning is commonly used in statistics and data analysis to simplify complex data sets and make them easier to interpret. Histogram (or any other plot with "bin" statistics) helps to give an overview of the sample distribution.

## Arguments

* Input (mandatory):
  - `x` - numeric sample on which the statistics are calculated; 
* Weights (optional):
  - `weights` - set of weights of the same size as the input sample; `null` (by default) means all weights are equal to `1.0` and the weighted count is equal to the normal one;
* Parameters (optional): 
  - `binsOption: BinsOption` - specifies either the number of bins or their width:
    - `BinsOption.byNumber(n: Int)` - values are divided into `n` bins (bins width are derived);
    - `BinsOption.byWidth(width: Double)` - values are divided into bins of width `width` (bins number is derived);
  - `binsAlign: BinsAlign` - specifies bins aligning:
    - `BinsAlign.center(pos: Double)` - bins are aligned by centering bin in `pos`;
    - `BinsAlign.boundary(pos: Double)` - bins are aligned by boundary between two bins in `pos`;
    - `BinsAlign.none()` - no aligning;

### Generalized signature

The specific signature depends on the function, but all functions related to "bin" statistic (which will be discussed further below - different variations of `statBin()`, `histogram()`) have approximately the same signature with the arguments above:

```
statBinArgs := 
   x, 
   weights = null, 
   binsOption = BinsOption.byNumber(20), 
   binsAlign = BinsAlign.center(0.0)
```

The possible types of `x` and `weights` depend on where a certain function is used. They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame` (`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name                 | type       | description                                              |
|----------------------|------------|----------------------------------------------------------|
| Stat.x               | Double[^1] | Center of bin                                            |
| Stat.count           | Int        | Number of observations in this bin                       |
| Stat.countWeighted   | Double     | Weighted count (sum of observations weights in this bin) |
| Stat.density         | Double     | Empirically estimated density in this bin                |
| Stat.densityWeighted | Double     | Weighted density                                         |

[^1]: TODO: will be changed to `T` generic type of sample in next versions.

## StatBin plots

In [1]:
%use dataframe(0.11.1)
%use kandy(0.4.5-dev-27)
@file:Repository("https://packages.jetbrains.team/maven/p/kds/kotlin-ds-maven")
@file:DependsOn("org.jetbrains.kotlinx:kotlin-statistics-jvm:0.0.0-dev-4")

In [2]:
// to generate the data we use a standard java math library
// https://commons.apache.org/proper/commons-math/
import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution

// generate sample from normal distribution
val depthList = NormalDistribution(500.0, 100.0).sample(1000).toList()
// generate sample from uniform distribution
val coeffList = UniformRealDistribution(0.0, 1.0).sample(1000).toList()

In [3]:
// gather them into the DataFrame
val df = dataFrameOf(
    "depth" to depthList,
    "coeff" to coeffList
)

`df` has a signature

<table>
  <thead>
    <tr>
      <th>depth</th>
      <th>coeff</th>
    </tr>
  </thead>
</table>

In [4]:
df.head(5)

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1040187392"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i

Let's take a look at `StatBin` output DataFrame:

In [5]:
df.statBin("depth", "coeff", binsOption = BinsOption.byNumber(10))

<iframe onload="o_resize_iframe_out_2()" style="width:100%;" class="result_container" id="iframe_out_2" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1040187393"></table>

<p class="dataframe_description">DataFrame: rowsCount = 10, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; 

It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>count</th>
      <th>countWeighted</th>
      <th>density</th>
      <th>densityWeighted</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics. For `statBin`, each row corresponds to one bin. `Stat.x` is the column with the centers of the bins; `Stat.count` contains the number of observations in the bin. `Stat.countWeighted` - weighted version of `count`. There are also `Stat.density` and `Stat.densityWeighted`. They contain empirically estimated density (both normal and weighted) of sample in the points corresponding to the centers of bins.

`DataFrame` with "bin" statistics is called `StatBinFrame`

### `statBin` transform

`statBin(statBinArgs) { /*new plotting context*/ }` modifies a plotting context - instead of original data (no matter was it empty or not) new `StatBin` dataset (calculated on given arguments; inputs and weights can be provided as `Iterable` or as dataset column reference - by name as a `String`, as a `ColumnReference` or as a `DataColumn`) is used inside a new context (original dataset and primary context are not affected - you can add layers using initial dataset outside the `statBin` context). Since the old dataset is irrelevant, we cannot use references for its columns. But we can refer to the new ones. They are all contained in the `Stat` group and can be called inside the new context:

In [6]:
plot {
    statBin(depthList, binsAlign = BinsAlign.center(500.0)) {
        // new `StatBin` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.count)
            fillColor = Color.RED
            alpha = 0.5
        }
    }
} 

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.411735619708878 331.49545619774625 L24.411735619708878 331.49545619774625 L50.10829942992876 336.49727371864776 L75.80486324014865 321.4918211559433 L101.50142705036856 306.4863685932388 L127.19799086058836 298.98364231188657 L152.89455467080828 258.9691021446747 L178.59111848102808 161.4336604870956 L204.287682291248 151.43002544529264 L229.9842461014679 113.91639403853145 L255.68080991168776 53.8945837877136 L281.3773737219077 16.380952380952408 L307.0739375321275 71.4009451108688 L332.7705013423474 111.41548527808072 L358.46706515256733 118.91821155943296 L384.16362896278724 203.94910941475828 L409.86019277300716 261.47001090512543 L435.55675658322707 303.9854598327881 L461.25332039344687 308.9872773536896 L486.9498842036668 313.98909487459105 L512.6464480138866 333.99636495819703 L512.6464480138866 344.0 L486.9498842036668 344.0 L461.25332039344687 344.0 L435.55675658322707 344.0 L409.86019277300716 344.0 L384.16362896278724 344.0 L358.46706515256733 344.0 L332.7705013423474 344.0 L307.0739375321275 344.0 L281.3773737219077 344.0 L255.68080991168776 344.0 L229.9842461014679 344.0 L204.287682291248 344.0 L178.59111848102808 344.0 L152.89455467080828 344.0 L127.19799086058836 344.0 L101.50142705036856 344.0 L75.80486324014865 344.0 L50.10829942992876 344.0 L24.411735619708878 344.0 Z" fill="rgb(238,102,102)" stroke-width="1.0" fill-opacity="0.5019607843137255">
 
 
 
 <path d="M24.411735619708878 331.49545619774625 L24.411735619708878 331.49545619774625 L50.10829942992876 336.49727371864776 L75.80486324014865 321.4918211559433 L101.50142705036856 306.4863685932388 L127.19799086058836 298.98364231188657 L152.89455467080828 258.9691021446747 L178.59111848102808 161.4336604870956 L204.287682291248 151.43002544529264 L229.9842461014679 113.91639403853145 L255.68080991168776 53.8945837877136 L281.3773737219077 16.380952380952408 L307.0739375321275 71.4009451108688 L332.7705013423474 111.41548527808072 L358.46706515256733 118.91821155943296 L384.16362896278724 203.94910941475828 L409.86019277300716 261.47001090512543 L435.55675658322707 303.9854598327881 L461.25332039344687 308.9872773536896 L486.9498842036668 313.98909487459105 L512.6464480138866 333.99636495819703 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 
 count
 
 
 
 
 x

### Histogram

Histogram is a statistical plot used for visualizing approximate representation of the distribution of numerical variable. It's a bar plot where each bar is representing a bin: its x coordinate is corresponding to bin range and y to count. So basically, we can build a histogram with `statBin` as follows:

In [7]:
val statBinBarsPlot = df.plot {
    statBin("depth") {
        bars {
            x(Stat.x)
            y(Stat.count)
        }
    }
    layout.title = "`statBin` + `bars`"
}
statBinBarsPlot

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `statBin` + `bars`
 
 
 
 
 count
 
 
 
 
 x

But we can do it even faster with `histogram(statBinArgs)` method:

In [8]:
val histogramPlot = plot {
    histogram(depthList)
    layout.title = "`histogram`"
}
histogramPlot

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `histogram`
 
 
 
 
 count
 
 
 
 
 x

Let's compare them:

In [9]:
plotGrid(listOf(statBinBarsPlot, histogramPlot))

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `statBin` + `bars`
 
 
 
 
 count
 
 
 
 
 x
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `histogram`
 
 
 
 
 count
 
 
 
 
 x

These two plots are identical. Indeed, `histogram` just uses `statBin` and `bars` and performs coordinates mappings under the hood. And of course we can customize histogram layer: `histogram()` optionally opens a new context, where we can configure bars (as in usual context opened by `bars { ... }`) - even change coordinate mappings from default ones. `StatBin` dataset of histogram is also can be accessed here.

In [10]:
df.plot {
    histogram(depth, binsAlign = BinsAlign.center(500.0)) {
        // change a column mapped on `y` to `Stat.density`
        y(Stat.density)
        // filling color depends on `density` statistic
        fillColor(Stat.density) {
            scale = continuous(Color.YELLOW..Color.RED)
        }
        borderLine.color = Color.BLACK
    }
}

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000
 
 
 
 
 
 
 0.0005
 
 
 
 
 
 
 0.0010
 
 
 
 
 
 
 0.0015
 
 
 
 
 
 
 0.0020
 
 
 
 
 
 
 0.0025
 
 
 
 
 
 
 0.0030
 
 
 
 
 
 
 0.0035
 
 
 
 
 
 
 0.0040
 
 
 
 
 
 
 0.0045
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 density
 
 
 
 
 depth
 
 
 
 
 
 
 
 
 density
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.001
 
 
 
 
 
 
 
 
 0.002
 
 
 
 
 
 
 
 
 0.003
 
 
 
 
 
 
 
 
 0.004

If we specify weights, `Stat.countWeighted` mapped to `y` by default:

In [11]:
import org.jetbrains.kotlinx.statistics.dataframe.stat.mean

df.plot {
    // count sample mean
    val mean = depth.mean()
    // add weighted histogram
    histogram(depth, coeff, binsOption = BinsOption.byNumber(10), binsAlign = BinsAlign.boundary(mean))
    // we can add other layers as well
    // let's add a vertical markline in the mean of sample
    vLine {
        xIntercept.constant(mean)
        tooltips() { line("Depth mean: ${String.format("%.2f", mean)}m") }
        color = Color.RED; width = 3.0
    }
    x.axis.name = "depth, m"
}

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted
 
 
 
 
 depth, m

### `histogram` plot

`histogram(statBinArgs)` and `DataFrame.histogram(statBinArgs)` is a family of functions for fast plotting a histogram.

In [12]:
histogram(depthList, binsAlign = BinsAlign.center(500.0))

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 x

In [13]:
df.histogram("depth")

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 depth

In case you want to provide inputs and weights using column selection DSL, it's a bit different from the usual one - you should assign `x` input and (optionally) `weight` throw invocation eponymous functions:

In [14]:
df.histogram(binsOption = BinsOption.byNumber(10)) {
    x(depth)
    weight(coeff)
}

200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 20
 
 
 
 
 
 
 40
 
 
 
 
 
 
 60
 
 
 
 
 
 
 80
 
 
 
 
 
 
 100
 
 
 
 
 
 
 120
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted
 
 
 
 
 depth

Histogram plot can be configured with `.configure {}` extension - it opens context that combines bars, `StatBin` and plot context; that means you can configure bars settings, mappings using `StatBin` dataset and any plot adjustments:

In [15]:
df.histogram(binsOption = BinsOption.byNumber(15)) {
    x(depth)
}.configure {
    // Bars + StatBin + PlotContext
    // can't add new layer
    x.limits = 100..900
    // can add bars mapping, incl. on `Stat.*` columns
    fillColor(Stat.count) { scale = continuous(Color.GREEN..Color.RED) }
    // can configure general plot adjustments
    layout {
        title = "Configured histogram plot"
        size = 600 to 350
    }
}

200
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 50
 
 
 
 
 
 
 100
 
 
 
 
 
 
 150
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Configured histogram plot
 
 
 
 
 count
 
 
 
 
 depth
 
 
 
 
 
 
 
 
 count
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50
 
 
 
 
 
 
 
 
 100
 
 
 
 
 
 
 
 
 150

## Grouped `staBin`

`statBin` can be applied for grouped data - statistics will be counted on each group independently but with equal bins. This application returns a new `GroupBy` dataset with the same keys as the old one but with `StatBin` groups instead of old ones.

In [16]:
// create two samples from normal distribution with different mean/std
val rangesA = NormalDistribution(500.0, 100.0).sample(5000).toList()
val rangesB = NormalDistribution(400.0, 80.0).sample(5000).toList()

In [17]:
// gather them into `DataFrame` with "A" and "B" keys in "category" column
val rangesDF = dataFrameOf(
    "range" to rangesA + rangesB,
    "category" to List(5000) { "A" } + List(5000) { "B" }
)
rangesDF.head(5)

<iframe onload="o_resize_iframe_out_3()" style="width:100%;" class="result_container" id="iframe_out_3" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1040187394"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i

It has the following signature:

<table>
  <thead>
    <tr>
      <th>range</th>
      <th>category</th>
    </tr>
  </thead>
</table>

In [18]:
// group it by "category"
val groupedRangesDF = rangesDF.groupBy { category }
groupedRangesDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1040187395"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) {
 if (df.c

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[range|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [19]:
groupedRangesDF.statBin { x(range) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1040187398"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) {
 if (df.c

After `statBin` applying it's still a `GroupBy` but with different signature of `group` - all groups have the same signature as usual `DataFrame` after `statBin` applying (i.e. `StatBinFrame`):

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: StaBinFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statBin` transformation within groups, the grouping keys did not change. Also, all bins centers matches - it helps to build grouped histogram.

The plotting process doesn't change much - we do everything the same. 

In [20]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
        }
    }
}

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000
 
 
 
 
 
 
 0.001
 
 
 
 
 
 
 0.002
 
 
 
 
 
 
 0.003
 
 
 
 
 
 
 0.004
 
 
 
 
 
 
 0.005
 
 
 
 
 
 
 0.006
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.818367630399187 343.30809071252577 L23.818367630399187 343.30809071252577 L48.89033355713519 342.61618142505154 L73.96229948387118 338.11877105646903 L99.03426541060719 325.3184492381958 L124.10623133734316 300.7556695328607 L149.17819726407916 244.0191079599739 L174.25016319081516 157.18449238195808 L199.32212911755116 93.87479257806612 L224.39409504428713 16.380952380952408 L249.4660609710231 41.63564137376176 L274.5380268977591 125.01071051440647 L299.6099928244952 196.62332176798915 L324.68195875123104 265.12234122793785 L349.7539246779671 311.4802634887112 L374.8258906047031 332.2375421129381 L399.89785653143906 341.9242721375773 L424.96982245817503 342.61618142505154 L450.041788384911 344.0 L475.11375431164697 344.0 L500.18572023838306 344.0 L500.18572023838306 344.0 L475.11375431164697 344.0 L450.041788384911 344.0 L424.96982245817503 344.0 L399.89785653143906 344.0 L374.8258906047031 344.0 L349.7539246779671 344.0 L324.68195875123104 344.0 L299.6099928244952 344.0 L274.5380268977591 344.0 L249.4660609710231 344.0 L224.39409504428713 344.0 L199.32212911755116 344.0 L174.25016319081516 344.0 L149.17819726407916 344.0 L124.10623133734316 344.0 L99.03426541060719 344.0 L73.96229948387118 344.0 L48.89033355713519 344.0 L23.818367630399187 344.0 Z" fill="rgb(71,71,71)" stroke-width="1.0" fill-opacity="0.10196078431372549">
 
 
 
 <path d="M23.818367630399187 343.30809071252577 L23.818367630399187 343.30809071252577 L48.89033355713519 342.61618142505154 L73.96229948387118 338.11877105646903 L99.03426541060719 325.3184492381958 L124.10623133734316 300.7556695328607 L149.17819726407916 244.0191079599739 L174.25016319081516 157.18449238195808 L199.32212911755116 93.87479257806612 L224.39409504428713 16.380952380952408 L249.4660609710231 41.63564137376176 L274.5380268977591 125.01071051440647 L299.6099928244952 196.62332176798915 L324.68195875123104 265.12234122793785 L349.7539246779671 311.4802634887112 L374.8258906047031 332.2375421129381 L399.89785653143906 341.9242721375773 L424.96982245817503 342.61618142505154 L450.041788384911 344.0 L475.11375431164697 344.0 L500.18572023838306 344.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 <path d="M23.818367630399187 344.0 L23.818367630399187 344.0 L48.89033355713519 344.0 L73.96229948387118 343.3298417002569 L99.03426541060719 339.3088919017984 L124.10623133734316 335.2879421033399 L149.17819726407916 320.20938035912053 L174.25016319081516 298.42923561747034 L199.32212911755116 260.90037083185763 L224.39409504428713 221.3610311470157 L249.4660609710231 157.36091352155117 L274.5380268977591 106.42888274107679 L299.6099928244952 93.36079589608664 L324.68195875123104 112.79538658863606 L349.7539246779671 150.99440967399187 L374.8258906047031 216.33484389894252 L399.89785653143906 258.88989593262835 L424.96982245817503 303.1203437156719 L450.041788384911 323.2250927079644 L475.11375431164697 333.2774672041107 L500.18572023838306 341.9895251007708 L500.18572023838306 344.0 L475.11375431164697 344.0 L450.041788384911 344.0 L424.96982245817503 344.0 L399.89785653143906 344.0 L374.8258906047031 344.0 L349.7539246779671 344.0 L324.68195875123104 344.0 L299.6099928244952 344.0 L274.5380268977591 344.0 L249.4660609710231 344.0 L224.39409504428713 344.0 L199.32212911755116 344.0 L174.25016319081516 344.0 L149.17819726407916 344.0 L124.10623133734316 344.0 L99.03426541060719 344.0 L73.96229948387118 344.0 L48.89033355713519 344.0 L23.818367630399187 344.0 Z" fill="rgb(71,71,71)" stroke-width="1.0" fill-opacity="0.10196078431372549">
 
 
 
 <path d="M23.818367630399187 344.0 L23.818367630399187 

As you can see, there are two areas because we have two groups of data. To distinguish them, we need to add mapping to the filling color from the key. This is easy - key is available in the context

In [21]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
            // can access "key." columns and create mapping from them
            fillColor(key.category)
            alpha = 0.6
        }
    }
}

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000
 
 
 
 
 
 
 0.001
 
 
 
 
 
 
 0.002
 
 
 
 
 
 
 0.003
 
 
 
 
 
 
 0.004
 
 
 
 
 
 
 0.005
 
 
 
 
 
 
 0.006
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M19.865713083991892 343.30809071252577 L19.865713083991892 343.30809071252577 L40.77699001450968 342.61618142505154 L61.68826694502745 338.11877105646903 L82.59954387554524 325.3184492381958 L103.510820806063 300.7556695328607 L124.42209773658077 244.0191079599739 L145.33337466709855 157.18449238195808 L166.24465159761633 93.87479257806612 L187.1559285281341 16.380952380952408 L208.06720545865187 41.63564137376176 L228.9784823891697 125.01071051440647 L249.88975931968747 196.62332176798915 L270.80103625020524 265.12234122793785 L291.712313180723 311.4802634887112 L312.62359011124084 332.2375421129381 L333.53486704175856 341.9242721375773 L354.4461439722764 342.61618142505154 L375.3574209027941 344.0 L396.26869783331193 344.0 L417.1799747638297 344.0 L417.1799747638297 344.0 L396.26869783331193 344.0 L375.3574209027941 344.0 L354.4461439722764 344.0 L333.53486704175856 344.0 L312.62359011124084 344.0 L291.712313180723 344.0 L270.80103625020524 344.0 L249.88975931968747 344.0 L228.9784823891697 344.0 L208.06720545865187 344.0 L187.1559285281341 344.0 L166.24465159761633 344.0 L145.33337466709855 344.0 L124.42209773658077 344.0 L103.510820806063 344.0 L82.59954387554524 344.0 L61.68826694502745 344.0 L40.77699001450968 344.0 L19.865713083991892 344.0 Z" fill="rgb(55,126,184)" stroke-width="1.0" fill-opacity="0.6">
 
 
 
 <path d="M19.865713083991892 343.30809071252577 L19.865713083991892 343.30809071252577 L40.77699001450968 342.61618142505154 L61.68826694502745 338.11877105646903 L82.59954387554524 325.3184492381958 L103.510820806063 300.7556695328607 L124.42209773658077 244.0191079599739 L145.33337466709855 157.18449238195808 L166.24465159761633 93.87479257806612 L187.1559285281341 16.380952380952408 L208.06720545865187 41.63564137376176 L228.9784823891697 125.01071051440647 L249.88975931968747 196.62332176798915 L270.80103625020524 265.12234122793785 L291.712313180723 311.4802634887112 L312.62359011124084 332.2375421129381 L333.53486704175856 341.9242721375773 L354.4461439722764 342.61618142505154 L375.3574209027941 344.0 L396.26869783331193 344.0 L417.1799747638297 344.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 <path d="M19.865713083991892 344.0 L19.865713083991892 344.0 L40.77699001450968 344.0 L61.68826694502745 343.3298417002569 L82.59954387554524 339.3088919017984 L103.510820806063 335.2879421033399 L124.42209773658077 320.20938035912053 L145.33337466709855 298.42923561747034 L166.24465159761633 260.90037083185763 L187.1559285281341 221.3610311470157 L208.06720545865187 157.36091352155117 L228.9784823891697 106.42888274107679 L249.88975931968747 93.36079589608664 L270.80103625020524 112.79538658863606 L291.712313180723 150.99440967399187 L312.62359011124084 216.33484389894252 L333.53486704175856 258.88989593262835 L354.4461439722764 303.1203437156719 L375.3574209027941 323.2250927079644 L396.26869783331193 333.2774672041107 L417.1799747638297 341.9895251007708 L417.1799747638297 344.0 L396.26869783331193 344.0 L375.3574209027941 344.0 L354.4461439722764 344.0 L333.53486704175856 344.0 L312.62359011124084 344.0 L291.712313180723 344.0 L270.80103625020524 344.0 L249.88975931968747 344.0 L228.9784823891697 344.0 L208.06720545865187 344.0 L187.1559285281341 344.0 L166.24465159761633 344.0 L145.33337466709855 344.0 L124.42209773658077 344.0 L103.510820806063 344.0 L82.59954387554524 344.0 L61.68826694502745 344.0 L40.77699001450968 344.0 L19.865713083991892 344.0 Z" fill="rgb(228,26,28)" stroke-width="1.0" fill-opacity="0.6">
 
 
 
 <path d="M19.865713083991892 344.0 L19.865713083991892 344.0 L40.77699001450968 344.0 L61.6882

The `histogram` layer also works. Moreover, if we have exactly one grouping key, a mapping from it to `fillColor` will be created by default.

In [22]:
groupedRangesDF.plot {
    histogram(range)
}

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

We can customize it like we used to. From the differences - access to `key` columns and we can customize the `position` of bars (within a single x-coordinate), for example - stack them:

In [23]:
groupedRangesDF.plot {
    histogram(column<Double>("range")) {
        fillColor(key.category) {
            scale = categorical(listOf(Color.GREEN, Color.ORANGE))
        }
        borderLine.width = 0.0
        width = 1.0
        // adjust position of bars
        position = Position.stack()
    }
}

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 200
 
 
 
 
 
 
 400
 
 
 
 
 
 
 600
 
 
 
 
 
 
 800
 
 
 
 
 
 
 1,000
 
 
 
 
 
 
 1,200
 
 
 
 
 
 
 1,400
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

Histogram plot for `GroupBy` (i.e. `GroupBy.histogram(statBinArgs)` extensions) works as well:

In [24]:
groupedRangesDF.histogram("range")

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

... and can be configured the same way:

In [25]:
groupedRangesDF.histogram(binsAlign = BinsAlign.center(500.0)) { x(range) }.configure {
    alpha = 0.6
    // make the bars from different groups overlap with each other
    position = Position.identity()
    // can access key column by name as `String`
    fillColor("category") { scale = categoricalColorBrewer(BrewerPalette.Qualitative.Dark2) }
}

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [26]:
rangesDF.plot {
    groupBy(category) {
        histogram(range)
    }
}

100
 
 
 
 
 
 
 
 
 200
 
 
 
 
 
 
 
 
 300
 
 
 
 
 
 
 
 
 400
 
 
 
 
 
 
 
 
 500
 
 
 
 
 
 
 
 
 600
 
 
 
 
 
 
 
 
 700
 
 
 
 
 
 
 
 
 800
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 100
 
 
 
 
 
 
 200
 
 
 
 
 
 
 300
 
 
 
 
 
 
 400
 
 
 
 
 
 
 500
 
 
 
 
 
 
 600
 
 
 
 
 
 
 700
 
 
 
 
 
 
 800
 
 
 
 
 
 
 900
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count
 
 
 
 
 range
 
 
 
 
 
 
 
 
 category
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B